# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'lstm'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/clean_data.csv', 'r') as f:
        for d in list(csv.reader(f))[1:]:
            tweets.append(d[1])  # tweet
            labels.append(d[2])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'wb') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 55132
Labels: 55132


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  0),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 0),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    # stop_words = set(stopwords.words('english'))
    # words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    # words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = parsed_text.strip('#')
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence, pos_string = preprocess(d)
        # sentence = preprocess(d)
        # pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 6*50 if MODEL == 'conv' else 300

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))

Optimal all length: 19
Optimal sentence length: 19
Optimal pos sentence length: 19


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[0]))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [14]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)[0]
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(ft_model[w])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [15]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)

Exception ignored in: <function tqdm.__del__ at 0x7f04909f54c0>
Traceback (most recent call last):
  File "/home/karol/Dokumenty/Phase_2/venv/lib/python3.8/site-packages/tqdm/std.py", line 1135, in __del__
    self.close()
  File "/home/karol/Dokumenty/Phase_2/venv/lib/python3.8/site-packages/tqdm/notebook.py", line 288, in close
    self.disp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


In [16]:
wordtoken_features[0]

.05313821,  0.06321698, -0.03605764,
         0.10258007,  0.08100326,  0.07594081, -0.03922767, -0.03663611,
        -0.03327654,  0.12164076, -0.02289039, -0.09888647, -0.00628894,
         0.06339369,  0.08535896, -0.01850916,  0.1072917 , -0.07552885,
         0.03384264, -0.02119819, -0.10083826, -0.05661517,  0.0373754 ,
         0.06369513,  0.00443319, -0.01583502, -0.08481843,  0.0275021 ,
        -0.04105359,  0.08047359, -0.00491942,  0.05498312,  0.0717987 ,
        -0.07345515, -0.06859258, -0.0044669 , -0.10881177, -0.0106741 ,
        -0.02166576, -0.01816474,  0.04195465,  0.08615237,  0.13905346,
        -0.09469645,  0.04564467,  0.02457125, -0.05742387, -0.08573598,
         0.00843078,  0.00487886,  0.05301751, -0.04016562, -0.0206612 ,
         0.06091737, -0.00428687, -0.0995879 ,  0.06112354, -0.06491072,
         0.08497535,  0.07196878,  0.04080458,  0.01789756, -0.07999384,
        -0.00494165,  0.00711435,  0.09984791, -0.02160177,  0.05840885,
        -0.008

### Supervised fastText pos training

In [17]:
if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
    with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[1]))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [18]:
def get_pos_fts(data, length):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = preprocess(d)[1]
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    
    ft_matrices = []
    for pt in pos_tags:
        ft_matrix = []
        for t in pt:
            ft_matrix.append(ft_pos_model[t])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [19]:
pos_features = get_pos_fts(tweets, opt_pos_length)

In [20]:
pos_features[0]

  2.36194078e-02,  3.94937098e-02, -1.33664683e-02, -2.38217344e-03,
        -5.66337816e-03, -1.67813525e-02,  2.32206974e-02,  1.95275526e-02,
         1.45623069e-02, -1.03388773e-02,  2.42875889e-02,  2.23896489e-03,
         1.46114745e-03, -1.26153640e-02, -3.43660191e-02,  1.29799731e-02,
         2.36207917e-02,  4.42393031e-03, -1.66111775e-02,  1.89996362e-02,
        -1.76787861e-02, -4.19484451e-03,  2.93791424e-02,  1.40571303e-03,
        -3.30645405e-02, -5.34949638e-03, -6.24877252e-02,  5.40951951e-05,
        -1.62039902e-02,  1.22539059e-04, -6.89824694e-04, -6.74945815e-03,
         8.58627446e-03, -8.17644969e-03,  7.60170585e-03, -5.64159863e-02,
        -6.36072597e-03,  3.46803851e-02,  1.54401399e-02,  4.78678755e-02,
         6.33809939e-02,  1.17549775e-02, -1.86548289e-02, -2.26582475e-02,
         8.05687439e-03,  1.74643807e-02, -2.40706187e-02,  6.68099103e-03,
         5.03446236e-02, -8.28702301e-02, -9.58519464e-04,  2.19015907e-02,
        -7.55408139

In [21]:
np.array(wordtoken_features).shape

(55132, 19, 300)

In [22]:
np.array(pos_features).shape

(55132, 19, 300)

### All features

In [23]:
# #Now join them all up
# features = np.array(wordtoken_features)
features = np.concatenate([wordtoken_features, pos_features], axis=1)

In [24]:
features.shape

(55132, 38, 300)

In [25]:
features[0]


array([[ 8.8309618e-03, -3.1371165e-02,  1.7607152e-02, ...,
         9.4437562e-03,  1.4775164e-02,  1.1288437e-02],
       [ 9.0225087e-04, -3.0532095e-03,  1.8441859e-03, ...,
         2.3376469e-03,  3.9488346e-05,  1.6918076e-03],
       [-3.8287524e-02,  1.1269204e-01, -7.1831010e-02, ...,
        -2.6748180e-02, -4.3440558e-02, -3.5387807e-02],
       ...,
       [ 5.3222864e-03,  4.5493752e-02, -3.0021299e-02, ...,
        -6.2045576e-03, -1.8905648e-03, -8.4057659e-02],
       [ 1.6812164e-02, -2.7392833e-02,  2.3578398e-02, ...,
        -1.3578662e-02, -8.0203256e-03, -8.6662835e-03],
       [ 4.2647731e-02, -9.9409461e-02,  6.3191712e-02, ...,
        -5.7244174e-02, -2.2195261e-02, -5.7007175e-02]], dtype=float32)

## Save features & labels

In [26]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': np.array(wordtoken_features).shape[1]}, serialized=True)
archive.dump()
del archive